# Section 1: Destacados 

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
from itertools import zip_longest as zip
from itertools import count
import numpy as np
import codecs

## A. Writing Json file with format 
def write_json_UTF8(jsonDir, name_file):
    with open(name_file, 'w') as outfile:
        json.dump(jsonDir, outfile, ensure_ascii=False)
    with codecs.open(name_file, 'r', encoding = "iso-8859-1") as file:
          lines = file.read()
    with codecs.open(name_file, 'w', encoding = 'utf8') as file:
          file.write(lines)
    print("CREATED: ", name_file)

## B. Fixing numbers 
def repair_num(value):
    try:
        aux = float(value)

    except:
        "Existe algun signo"
        aux = ""
        for letra in str(value):
            if letra.isdigit():
                aux = aux+letra
            elif letra == ".":
                aux = aux+letra
        if aux == "":
            pass
        
    return float(aux)

## C. Formating Json
def DataToJson(df, varible_v, variable_meaning, reg_name, toolTip_v, toolTip_mening, norm=False):

    globalDir = {}
    anioInit = -999
    listData = []
    dirData = {}
    initDir = True

    df_norm = None
    if norm:
        df_copy = df.copy()
        df_norm = (df_copy[varible_v]-df_copy[varible_v].min()) / \
            (df_copy[varible_v].max()-df_copy[varible_v].min())

    for i, anio, mes, var_value in zip(count(), df["anio"], df["mesSTR"], df[varible_v]):
        try:
            anio = int(anio)
        except:
            print("set:", anio)
            listData.append(dirData)
            break

        # Si todas son nan no sirve
        L = 0

        for index, v_reg in enumerate(varible_v):
            if str(df[v_reg].values[i]) == "nan":
                L = L+1
        if L == len(varible_v):
            print("allNone", anio)
            continue

        if anioInit < anio:
            if anioInit > 0:
                listData.append(dirData)
                dirData = {}
                initDir = True

        if initDir:
            dirData["anio"] = anio
            dirData["regiones"] = {}
            for index, v_reg in enumerate(varible_v):
                dirData["regiones"][reg_name[index]] = {"meses": {}}

            initDir = False

        for index, v_reg in enumerate(varible_v):
            dirData["regiones"][reg_name[index]]["meses"][mes] = {}

        for index, v_reg in enumerate(varible_v):
            if norm:
                dirData["regiones"][reg_name[index]]["meses"][mes][variable_meaning[index]] = round(
                    float(df_norm[v_reg].values[i]), 4)
                if (df_norm[v_reg].values[i]) == "nan":
                    del dirData["regiones"][reg_name[index]]["meses"][mes]
                    break

            else:
                # si no hay data
                data_value = repair_num(df[v_reg].values[i])
                """
                try:
                    data_value=float(df[v_reg].values[i])
                except:
                    "Existe algun signo"
                    aux=""
                    for letra in str(df[v_reg].values[i]):
                        if letra.isdigit():
                            aux=aux+letra
                        elif letra ==".":
                            aux=aux+letra
                            
                    data_value=float(aux)
                """
                if str(data_value).lower() == "nan" or len(str(data_value).lower()) == 0:
                    data_value = 0
                if data_value - int(data_value) > 0.0001:
                    data_value = round(data_value, 1)
                else:
                    data_value = int(data_value)
                dirData["regiones"][reg_name[index]
                                    ]["meses"][mes][variable_meaning[index]] = data_value
                if (df[v_reg].values[i]) == "nan":
                    del dirData["regiones"][reg_name[index]]["meses"][mes]
                    break

            if len(toolTip_v[index]) > 0:

                toolTip_values = [str(df[x].values[i])
                                  for x in toolTip_v[index]]
                tip = {}
                #print(index, v_reg, i, df[v_reg].values[i])
                data_value = repair_num(df[v_reg].values[i])

                """
                try:
                    data_value=float(df[v_reg].values[i])
                except:
                    "Existe algun signo"
                    aux=""
                    for letra in str(df[v_reg].values[i]):
                        if letra.isdigit():
                            aux=aux+letra
                        elif letra ==".":
                            aux=aux+letra
                            
                    data_value=float(aux)
                
                """

                if str(data_value).lower() == "nan" :
                    data_value = "No Data"
                else:
                    if data_value - int(data_value) > 0.0001:
                        data_value = round(data_value, 1)
                    else:
                        data_value = int(data_value)
                tip[variable_meaning[index]] = data_value

                # add data to tooltip
                tip["Fecha: "] = str(int(df["anio"].values[i])) + \
                    "/ " + df["mesSTR"].values[i]

                for indexTool, name in enumerate(toolTip_mening[index]):
                    if str(toolTip_values[indexTool]).lower() == "nan":
                        tip[name] = "-"
                    else:
                        tip[name] = toolTip_values[indexTool]

                dirData["regiones"][reg_name[index]
                                    ]["meses"][mes]["tooltip"] = tip
            else:

                tip = {}
                data_value = repair_num(df[v_reg].values[i])
                if str(data_value).lower() == "nan" :
                    data_value = "No Data"
                else:
                    if data_value - int(data_value) > 0.0001:
                        data_value = round(data_value, 1)
                    else:
                        data_value = int(data_value)
                tip[variable_meaning[index]] = data_value

                tip["Fecha: "] = str(int(df["anio"].values[i])) + \
                    "/ " + df["mesSTR"].values[i]
                
                dirData["regiones"][reg_name[index]]["meses"][mes]["tooltip"] = tip
        anioInit = anio

    return listData

## D. Creating image to download 
sns.set()
def create_image(df, varible_v, variable_meaning, save_dir):
    print("img", varible_v, variable_meaning)
    objetive_var = varible_v

    plt.close()
    plt.clf()
    
    df['date'] = df["Mes"].map(str) + '-' + df["anio"].map(str)
    for value in df['date'] :
        df['date'] = pd.to_datetime(
        df['date'], format='%m-%Y').dt.strftime('%m-%Y')

    date = []
    val = []
    for i, value in enumerate(df[objetive_var]):
        if str(value) == "nan":
            continue
        else:
            value = repair_num(value)
            val.append(value)
            print((df['date'].values[i]))
            date.append(df['date'].values[i])

    df = pd.DataFrame(columns=['date', objetive_var])
    df['date'] = date
    df[objetive_var] = val

    fig, ax = plt.subplots()
    sns.set_style("darkgrid")

    print(len(df['date']), len(df[objetive_var]))

    ax = sns.lineplot(x=range(0, len(df['date'])), y=df[objetive_var])

    """
    max_xticks = 11
    xloc = plt.MaxNLocator(max_xticks)
    print(xloc)
    ax.xaxis.set_major_locator(xloc)
    """

    names = []
    print(len(df['date']))

    ran = int((len(df['date'])+.5)/10)
    if ran == 0:
        ran = 1
    if len(df['date']) / ran < 10:
        ran = 1
    names = [df['date'].values[d] for d in range(0, len(df['date']), ran)]
    ticklabels = names
    ax.set_xticks(range(0, len(df['date'])+1, ran))
    ax.set_xticklabels(ticklabels)
    fig.autofmt_xdate()
    plt.title(variable_meaning)
    plt.ylabel(variable_meaning)
    plt.savefig(save_dir, dpi=100)
    plt.show()
    
def naming_image(x):
    return global_dir + "/img/" + init_data_name + x + '.png'

def naming_file(y):
    return global_dir + "/" + init_data_name + y + '.json'    

## E. Reading Data from API 
file='Destacados.xlsx'
global_dir="./data"
norm=False# no normalizada
init_data_name="Destacado_"
link_data = "https://smartdata-demo.sfo2.digitaloceanspaces.com/data/Destacados.xlsx"
link_img = "https://smartdata-demo.sfo2.digitaloceanspaces.com/data/img/"
link_glosario = "https://smartdata-demo.sfo2.digitaloceanspaces.com/data/"
info_header=[ {"titulo":None,"Fuente":None, "LinkDatosOriginales":None, 
               "comment":None, "linkPlot":None,"glosario":None}]

## Processing Data 

### 1) Destacado_01_PIB_Construccion.json

In [46]:
sheetname = 'PIB Construcción'
full_file_name = '01_PIB_Construccion'
info_header[0]["titulo"]="PIB Construcción ($MM)"
info_header[0]["Fuente"]="Banco Central"
info_header[0]["LinkDatosOriginales"]=link_data
info_header[0]["comment"]="PIB de la Construcción. Banco Central"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=='PIB Construcción':
    df = pd.read_excel( file, sheet_name = sheetname )
    data=[["Construcción"]]
    dataCorto=[["PIB Cont."]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header,jsonDir ]
    name_file= global_dir + "/" + init_data_name + full_file_name + '.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_01_PIB_Construccion.json


### 2) Destacado_02_Porc_PIB_Construccion.json

In [47]:
sheetname = 'PIB Construcción'
full_file_name = '02_Porc_PIB_Construccion'
info_header[0]["titulo"]="PIB Contrucción (%)"
info_header[0]["Fuente"]="Banco Central"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"]="Participación del PIB de construcción, % en relación PIB Nacional. Banco Central"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=='PIB Construcción':
    df = pd.read_excel( file, sheet_name = sheetname )
    data=[["porcentaje"]]
    dataCorto=[["PIB %"]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header,jsonDir ]
    name_file= global_dir+"/" + init_data_name + full_file_name+'.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8, encoding = 'utf-8')
    with open(name_file , 'w') as outfile:
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_02_Porc_PIB_Construccion.json


### 3) Destacado_03_IMACON.json

In [48]:
sheetname = "IMACON"
full_file_name = '03_IMACON'
info_header[0]["titulo"]='IMACON Variación Mensual'
info_header[0]["Fuente"]="CCHC"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"]="Índice Mensual de la Construcción. Variación mensual (%). CChC"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=="IMACON":
    df = pd.read_excel( file, sheet_name = sheetname )
    data=[["IMACON varMensual"]]
    dataCorto=[["Var Mes"]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header,jsonDir ]  
    name_file= global_dir+"/" + init_data_name + full_file_name + '.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)
    with open(name_file , 'w') as outfile:
        json.dump(jsonUTF8, outfile,ensure_ascii=False)    
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_03_IMACON.json


### 4) Destacado_04_Perm_Edificacion_m2.json

In [49]:
sheetname = "m2"
full_file_name = '04_Perm_Edificacion_m2'
info_header[0]["titulo"]='Permisos Edificación Acumulados a la fecha (m2)'
info_header[0]["Fuente"]="CCHC"
info_header[0]["LinkDatosOriginales"]=link_data
info_header[0]["comment"]="Permisos de edificación acumulados a la fecha en m2 total (habitacional y no habitacional). CChC"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=="m2":
    df = pd.read_excel( file, sheet_name = sheetname )
    data=[["acumulado Mill. M2"]]
    dataCorto=[["m2"]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header,jsonDir ]
    name_file= global_dir+"/" + init_data_name + full_file_name + '.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_04_Perm_Edificacion_m2.json


### 5) Destacado_05_Por_Certificacion_Sustentable.json

In [50]:
sheetname = "%_Certificacion"
full_file_name = '05_Por_Certificacion_Sustentable'
info_header[0]["titulo"]='Edificaciones con Certificación Sustentable (%)'
info_header[0]["Fuente"]="LEED, CES, otros"
info_header[0]["LinkDatosOriginales"]=link_data
info_header[0]["comment"]="Edificaciones con Certificación Sustentable, % respecto al total de edificaciones en m2. LEED, CES"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=="%_Certificacion":
    df = pd.read_excel( file, sheet_name = sheetname )
    data=[["%"]]
    dataCorto=[["Porc."]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header,jsonDir ]
    name_file= global_dir+"/" + init_data_name + full_file_name + '.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_05_Por_Certificacion_Sustentable.json


### 6) Destacado_06_Por_Trabajadores_Empleo_Nacional.json

In [51]:
sheetname = 'Trabajadores_%_Nacional'
full_file_name = '06_Por_Trabajadores_Empleo_Nacional'
info_header[0]["titulo"]="Participación empleo sectorial en total nacional (%)"
info_header[0]["Fuente"]="NENE del INE y SII"
info_header[0]["LinkDatosOriginales"]=link_data
info_header[0]["comment"]="Participación empleo sector construcción en total nacional %. INE y SII"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=""

if sheetname=='Trabajadores_%_Nacional':
    df = pd.read_excel(file, sheet_name=sheetname )
    data=[["Participacion"]]
    dataCorto=[["Porcentaje"]]
    for index,lista in enumerate(data):
        toolTip_v=[[]]
        toolTip_mening=[[]]
        reg_name=["PAIS"]
        for index2,varname in enumerate(lista):
            if index2==0:
                varible_v=[varname ]
                variable_meaning=[dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening,norm)
    jsonDir = [info_header,jsonDir ]
    name_file= global_dir + "/" + init_data_name + full_file_name + '.json'
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)

set: None
CREATED:  ./data/Destacado_06_Por_Trabajadores_Empleo_Nacional.json
